In [1]:
#IMPORT PACKAGE
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [2]:
#INPUT DATA
df = pd.read_excel("persatuan data berharga 2.xlsx")
predictors = ['Kecamatan','BR', 'Luas', 'Fasilitas','Lantai','Unit']
label = 'Harga'

In [3]:
#MENAMPILKAN INFO DATAFRAME
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2875 entries, 0 to 2874
Data columns (total 7 columns):
Kecamatan    2875 non-null object
BR           2875 non-null int64
Luas         2875 non-null float64
Fasilitas    2875 non-null object
Lantai       2875 non-null object
Unit         2875 non-null object
Harga        2875 non-null int64
dtypes: float64(1), int64(2), object(4)
memory usage: 157.3+ KB


In [4]:
#LABEL ENCODER
le = dict()
for column in df.columns:
    if df[column].dtype == np.object:
        le[column] = LabelEncoder()
        df[column] = le[column].fit_transform(df[column])

In [5]:
#INFO DATAFRAME
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2875 entries, 0 to 2874
Data columns (total 7 columns):
Kecamatan    2875 non-null int64
BR           2875 non-null int64
Luas         2875 non-null float64
Fasilitas    2875 non-null int64
Lantai       2875 non-null int64
Unit         2875 non-null int64
Harga        2875 non-null int64
dtypes: float64(1), int64(6)
memory usage: 157.3 KB


In [47]:
print (df)

      Kecamatan  BR   Luas  Fasilitas  Lantai  Unit     Harga
0             0   2   36.0          0       0     0   2700000
1             0   2   36.0          0       1     0   3000000
2             0   1   25.0          1       1     0   2000000
3             0   1   25.0          1       2     0   1700000
4             0   2   36.0          0       2     0   3500000
5             0   1   30.0          0       2     0   2850000
6             0   2   36.0          0       0     0   3650000
7             0   2   36.0          1       0     0   3500000
8             0   2   36.0          0       0     0   3500000
9             0   1   25.0          1       0     2   2700000
10            0   2   36.0          0       0     0   3500000
11            0   2   36.0          0       0     0   3500000
12            0   2   72.0          0       1     0   5500000
13            0   2  135.0          0       2     0   2000000
14            0   2  135.0          0       2     0   2000000
15      

In [6]:
#MEMBAGI DATA TRAIN DAN TEST
df_train, df_test, y_train, y_test = train_test_split(df[predictors], df[label], test_size=0.25, random_state=42)

In [49]:
#MEMBUAT MODEL TRAIN PARAMETER DEFAULT
def evaluatetrain(model, X=df_train, y=y_train):
    predictions = model.predict(X=df_train)
    errors = abs(predictions - y_train)
    mape = 100 * np.mean(errors / y_train)
    accuracy = 100 - mape
    print('Model Performance Training')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_modeltrain = RandomForestRegressor(random_state = 42)
base_model.fit(X=df_train, y=y_train)
base_accuracytrain = evaluatetrain(base_model, X=df_train, y=y_train)

Model Performance Training
Average Error: 682831.2026 degrees.
Accuracy = 89.49%.


In [50]:
#MEMBUAT MODEL TEST PARAMETER DEFAULT
def evaluatetest(model, X=df_test, y=y_test):
    predictionstest = model.predict(X=df_test)
    errorstest = abs(predictionstest - y_test)
    mapetest = 100 * np.mean(errorstest / y_test)
    accuracytest = 100 - mapetest
    print('Model Performance Testing')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errorstest)))
    print('Accuracy = {:0.2f}%.'.format(accuracytest))
    
    return accuracytest

base_modeltest = RandomForestRegressor(random_state = 42)
base_model.fit(X=df_test, y=y_test)
base_accuracytest = evaluatetest(base_model, X=df_test, y=y_test)

Model Performance Testing
Average Error: 568424.3797 degrees.
Accuracy = 91.31%.


In [16]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_features': [3,'sqrt','auto'],
    'n_estimators': [10, 50, 75, 100, 200, 300, 1000, 1500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)

In [17]:
# Fit the grid search to the data
grid_search.fit(X=df_train, y=y_train)
grid_search.best_params_

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  3.3min finished


{'bootstrap': True, 'max_features': 3, 'n_estimators': 300}

In [17]:
#EVALUATE THE BEST MODEL SEARCH GRID TRAINING DATA

In [51]:
best_gridtrain = RandomForestRegressor(random_state = 42, max_features=3, n_estimators=300  )
best_gridtrain.fit( X=df_train, y=y_train)
grid_accuracytrain = evaluatetrain(best_gridtrain, X=df_train, y=y_train)

Model Performance Training
Average Error: 641922.6056 degrees.
Accuracy = 90.08%.


In [54]:
print('Improvement of training {:0.2f}%.'.format( 100 * (grid_accuracytrain - base_accuracytrain) / base_accuracytrain))

Improvement of training 0.65%.


In [20]:
#EVALUATE THE BEST MODEL SEARCH GRID TESTING DATA

In [52]:
best_gridtest = RandomForestRegressor(random_state = 42, max_features=3, n_estimators=300  )
best_gridtest.fit(X=df_test, y=y_test)
grid_accuracytest = evaluatetest(best_gridtest, X=df_test, y=y_test)

Model Performance Testing
Average Error: 520686.3254 degrees.
Accuracy = 92.12%.


In [55]:
print('Improvement of testing {:0.2f}%.'.format( 100 * (grid_accuracytest - base_accuracytest) / base_accuracytest))

Improvement of testing 0.89%.


In [32]:
from sklearn.externals import joblib
joblib.dump(best_grid, 'model.pkl') 

['model.pkl']

In [33]:
df_test.head(1)

,Kecamatan,BR,Luas,Fasilitas,Lantai,Unit
729,5,2,34.0,0,2,0


In [34]:
import json

In [35]:
data = json.dumps({'Kecamatan':5, 'BR':2, 'Luas':34, 'Fasilitas':0, 'Lantai':2, 'Unit':0})

In [36]:
data = np.array([5, 2, 34, 0, 2, 0]).reshape(1, -1)

In [37]:
data

array([[ 5,  2, 34,  0,  2,  0]])

In [38]:
best_grid.predict(data)[0]

4967489.414028184

In [39]:
data = {'Kecamatan':5, 'BR':2, 'Luas':34, 'Fasilitas':0, 'Lantai':2, 'Unit':0}
predict_request = [data['Kecamatan'], data['BR'], data['Luas'], data['Fasilitas'], data['Lantai'], data['Unit']]
predict_request = np.array(predict_request).reshape(1, -1)
     # predict_request = np.array(predict_request)
     # # query = pd.DataFrame(json_)
     # # query = pd.get_dummies(query_df)
prediction = best_grid.predict(predict_request)
prediction[0]

4967489.414028184

In [40]:
#VARIABLE OF IMPORTANCE

In [41]:
# Get numerical feature importances
importances = list(best_grid.feature_importances_)
#List Predictors
predictors_list = list(predictors)
# List of tuples with variable and importance
feature_importances = [(predictors, round(importance, 2)) for predictors, importance in zip(predictors_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: Luas                 Importance: 0.62
Variable: Kecamatan            Importance: 0.18
Variable: BR                   Importance: 0.12
Variable: Lantai               Importance: 0.04
Variable: Unit                 Importance: 0.02
Variable: Fasilitas            Importance: 0.01


[None, None, None, None, None, None]